<h1> Main.ipynb </h1>
<h2> 1. Import libraries and modules </h2>

In [3]:
import numpy as np
import matplotlib.pyplot as plt

In [172]:
(3e-5)*1000000

30.0

In [153]:
alfa = np.array([20.6, 20.87, 20.95, 20.85])
bravo = np.array([21.34, 21.1, 21.09, 21.15])

In [162]:
from scipy.stats import ttest_ind

esg_list_1 = [23.1, 25.3, 21.9, 24.0]
esg_list_2 = [27.8, 28.1, 26.5, 29.0]

# Perform Welch's t-test (assumes unequal variances)
t_stat, p_value = ttest_ind(alfa, bravo, equal_var=False)

print(f"T-statistic: {t_stat:.4f}")
print(f"P-value: {p_value:.4f}")

T-statistic: -3.6939
P-value: 0.0114


In [164]:
np.mean(alfa) + (np.std(alfa)*3.6) - np.mean(bravo)

np.float64(0.11920647652963012)

In [142]:
input = np.random.normal(loc=1, scale=0.05, size=800)
# plt.hist(input)
periodic_return = np.cumprod(input)[-1]-1
print(periodic_return)

0.7642997810766432


In [137]:
periodic_return = np.cumprod(input)[-1]-1
print(periodic_return)

-0.4784415511728596


In [143]:
all_drawdown = []

ind_drawdown = []
for i in range(0, len(input), 1):
    if input[i] < 1:
        ind_drawdown.append(input[i])
        if i == len(input) - 1:
            all_drawdown.append(ind_drawdown)
    else:
        if len(ind_drawdown) == 0:
            pass
        else:
            all_drawdown.append(ind_drawdown)
            ind_drawdown = []

In [144]:
prod_drawdowns = np.abs(np.array([np.cumprod(all_drawdown[i])[-1] for i in range(len(all_drawdown))])-1)


if -int(len(prod_drawdowns) *0.1) == 0:
    idx = len(prod_drawdowns)
else: 
    idx= -int(len(prod_drawdowns) *0.1)

In [93]:
# prod_drawdowns

In [165]:
np.sort(prod_drawdowns)

array([0.00055765, 0.00065186, 0.00131443, 0.00224123, 0.00241958,
       0.00264809, 0.00271611, 0.00354343, 0.00363497, 0.00379754,
       0.00388691, 0.00404224, 0.0044822 , 0.00674357, 0.00695579,
       0.00704376, 0.00812196, 0.00818611, 0.00967591, 0.01137663,
       0.01224702, 0.01266219, 0.01271212, 0.01375717, 0.01378681,
       0.013894  , 0.01393603, 0.01394104, 0.01400516, 0.01449965,
       0.01627234, 0.01737579, 0.01759621, 0.01760652, 0.01773273,
       0.01842683, 0.02013063, 0.02177753, 0.02186229, 0.02310503,
       0.02338289, 0.02439999, 0.0251179 , 0.02621608, 0.02643209,
       0.02676632, 0.02697518, 0.02761126, 0.02800064, 0.02801497,
       0.02806908, 0.02869508, 0.02877813, 0.03027609, 0.03148622,
       0.0317162 , 0.03272412, 0.03547346, 0.03551046, 0.03643106,
       0.03663314, 0.03683004, 0.03712301, 0.03720642, 0.03732249,
       0.03784508, 0.03868294, 0.03868781, 0.04043775, 0.04100602,
       0.04197488, 0.04277854, 0.04504674, 0.04533217, 0.04557

In [145]:
alfa = np.mean(np.sort(prod_drawdowns)[::-1][:idx])

In [146]:
periodic_return/alfa

np.float64(9.719990478793084)

In [147]:
def _sterling_ratio(returns: list):
    """
    Sterling ratio calculator
    """
    
    periodic_return = np.cumprod(returns)[-1] -1

    all_drawdown = []
    ind_drawdown = []
    for i in range(0, len(input), 1):
        if input[i] < 1:
            ind_drawdown.append(input[i])
            if i == len(input) - 1:
                all_drawdown.append(ind_drawdown)
        else:
            if len(ind_drawdown) == 0:
                pass
            else:
                all_drawdown.append(ind_drawdown)
                ind_drawdown = []

    prod_drawdowns = np.abs(np.array([np.cumprod(all_drawdown[i])[-1] for i in range(len(all_drawdown))])-1)


    if -int(len(prod_drawdowns) *0.1) == 0:
        idx = len(prod_drawdowns)
    else: 
        idx= -int(len(prod_drawdowns) *0.1)
    avg_drawdown = np.mean(np.sort(prod_drawdowns)[::-1][:idx])
    print(periodic_return)
    print(avg_drawdown)
    sterling_ratio = periodic_return / avg_drawdown

    return sterling_ratio

In [151]:
1/18*3.5

0.19444444444444442

In [148]:
_sterling_ratio(input)

0.7642997810766432
0.0786317417433875


np.float64(9.719990478793084)

In [1]:
import numpy as np
import pandas as pd

from src.Data_Retriever import DataRetrieverNew as DatRet

from src.Optimization.Markowitz_PT import MarkowitzPT as MPT

from src.Optimization.Environment import PortfolioEnvironment as PorEnv
from src.Optimization.RLModelCompilation import RL_Model as RLM
# from src.Optimization.NeuralNet import CustomCNNExtractor 

from src.Result.Menchero_OGA import MencheroOGA as MOGA
from src.Result.IndPortResults import GenerateResult as GR
from src.Result.OverviewResults import ResultConveyor as RC

import time

import warnings
warnings.filterwarnings('ignore')

<h2> 2. Define operating variables </h2>

In [2]:
trading_n = 400
history_usage = 504
n_sectors = 6
n_stocks_per_sector = 4

# For RL algorithm
history_usage_RL = 40
rolling_reward_window = 40

<h2> 3. Define stock dataframe </h2>

In [3]:
ticker_df =  pd.DataFrame({
    "Petroleum": ["EQNR.OL", "SUBC.OL", "BWO.OL"],
    "Food": ["ORK.OL", "MOWI.OL", "LSG.OL"],
    "Materials": ["NHY.OL", "YAR.OL", "RECSI.OL"],
    "Technologies": ["TEL.OL", "NOD.OL", "ATEA.OL"],
    "Financial": ["STB.OL", "DNB.OL", "AKER.OL"],
    "Shipping":["SNI.OL", "BELCO.OL", "ODF.OL"]
})

<h2> 4. Define ESG array </h2>

In [4]:
esg_scores = np.array([36.6, 17.9, 18, 
                18, 23.2, 29.2, 
                15.7, 25.4, 25.6, # Del this
                19.8, 13.8, 18.1, 
                17.3, 14, 12.3, 
                21.2, 26.8, 24.9
                ])

<h2> 5.-7. (Non-necessary for reproducability) </h2>
<h3>Data retrieval</h3>
<h3>Benchmark optimization (MPT)</h3>
<h3>RL optimization x4</h3>
<br>
All of the data has been generated an stored in csvs. Therefore, I can comment out the code lines below.

In [5]:
# Retrieve data from yf API: y-m-d
data = DatRet("2006-07-01", "2025-03-31", ticker_df)
# In function below, set log=True to check for data availability
data.retrieve_data()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

--- Data retrieved successfully ---


In [6]:
# Generate benchmark weights thorugh MPT using Sharpe ratio
# # benchmark = MPT(history_usage, trading_n)
# IMPORTANT: In order to see  the effect of the weights, algo exclude last observation from optimization
# # benchmark.frequency_optimizing()

In [7]:
# objectives = ["Return", "Sharpe", "Sortino", "Sterling", "Return", "Sharpe", "Sortino", "Sterling"]
# esg_compliancy = [True, True, True, True, False, False, False, False]
# # objectives = ["Sharpe"]
# # esg_compliancy = [True]

# for i in range(len(objectives)):
#     reinforcement = RLM(esg_scores, 
#                         objective=objectives[i],
#                         history_usage=history_usage_RL,
#                         rolling_reward_window=rolling_reward_window,
#                         total_timesteps=200,
#                         esg_compliancy=esg_compliancy[i],
#                         )
#     reinforcement.train_model()
#     reinforcement.test_model()

<h2> 8. Attribution analysis </h2>

In [8]:
# paths = ["Return_esg_True", "Sharpe_esg_True",
#          "Sortino_esg_True","Sterling_esg_True",
#          "Return_esg_False", "Sharpe_esg_False",
#          "Sortino_esg_False","Sterling_esg_False",]

# analysis_list = []
# for i in range(len(paths)):
#     att_anal = GR(paths[i],
#             n_sectors, n_stocks_per_sector,
#             trading_n,
#             esg_scores, 
#             ticker_df.columns)
#     att_anal.friple_frequency_analysis()
#     analysis_list.append(att_anal)

<h2> 9. Overview  Result Analysis </h2>

In [9]:
# theta = RC(analysis_list, trading_n)
# theta.convey_results()

<h1> 10. Ad-hoc </h1>

In [10]:
# ret = pd.read_csv("Data/Input/StockReturns.csv")
# weights = pd.read_csv("Data/RL_weights_Sharpe_esg_false.csv")

# train_set = ret.iloc[:int(0.85*len(ret))]
# test_set = ret.iloc[int(0.85*len(ret)):]

In [11]:
# import pandas as pd
# import numpy as np
# import yfinance as yf

In [12]:
start_date = "2006-07-01"
end_date = "2024-03-31"

In [13]:
# class DataRetrieverNew:

#     def __init__(self, start_date, end_date, ticker_df):
#         """
#         doc string
#         """
#         self.start_date = start_date
#         self.end_date = end_date
#         self.ticker_list = np.array(ticker_df).flatten()

#         self.master_daterange = pd.Series(np.repeat(pd.date_range(start=self.start_date, end=self.end_date, freq="B"), 2), name="Date")
#         self.return_df = pd.DataFrame(index=self.master_daterange)
#         self.volume_df = pd.DataFrame(index=self.master_daterange)
#         self.rolling_return_df = pd.DataFrame(index=self.master_daterange)
#         self.rolling_volatility_df = pd.DataFrame(index=self.master_daterange)


#     def z_score(self, arr: np.array):
#         """
#         doc string
#         """
#         z_score_arr = (arr - np.mean(arr)) / np.std(arr)

#         return z_score_arr



#     def retrieve_data(self):
#         """
#         doc string
#         """
#         for i in range(0, len(self.ticker_list), 1):
#             stock_data = yf.download(self.ticker_list[i], start=start_date, end=end_date)

#             date_df = pd.DataFrame(index=self.master_daterange)
#             open = stock_data["Open"]
#             individual_df = pd.merge(date_df, open, on="Date", how="left")
#             inter_df = pd.merge(date_df, stock_data["Close"], on="Date", how="left")
#             individual_df[self.ticker_list[i]].iloc[1::2] = inter_df[self.ticker_list[i]].iloc[::2]
#             return_series = individual_df.interpolate().ffill().bfill()
#             vol_series = pd.merge(date_df, stock_data["Volume"], on="Date", how="left")

#             self.return_df[self.ticker_list[i]] = return_series[self.ticker_list[i]]
#             self.volume_df[self.ticker_list[i]] = self.z_score(vol_series[self.ticker_list[i]])
#             self.rolling_return_df[self.ticker_list[i]] = return_series[self.ticker_list[i]].rolling(40).mean().bfill()
#             self.return_df[self.ticker_list[i]] = return_series[self.ticker_list[i]].rolling(40).std().bfill()
        
#         master_df = pd.concat([self.return_df, self.volume_df, self.rolling_return_df, self.rolling_volatility_df], axis = 1)
#         master_df.to_csv("Mix.csv", index=False)

#         print("--- Data retrieved successfully ---")

In [14]:
# hotel = DataRetrieverNew(start_date,end_date, ["EQNR.OL", "SNI.OL"] )
# hotel.retrieve_data()

In [27]:
6%2

0

In [ ]:
for step in range(400):
    action, _ = self.model.predict(obs, deterministic=True)
    weights = action / np.sum(action+ 1e-8)
    weights_history_test.append(weights)

    obs, _ done, _ = test_env.step(action)
    if (step + 1) & self.retrain_interval == 0 and (step + 1) < 400:
        additional_data = self.valid

In [1]:
import numpy as np
import pandas as  pd

In [2]:
df = pd.read_csv("Data/StockReturns.csv")

In [25]:
halogen = []
for i in range(len(df)):
    ret = df.iloc[i]
    check = [ret[int(i*3):int((1+i)*3)] for i in range(4)]
    
    for j in range(0,len(check),1):
        if len(check[j]) != 3:
            print("Nei")
    halogen.append(check)

In [17]:
len(halogen[0][1])

3